In [5]:
# Task 2: Dynamic Programming Practice
# Author: Jaydine Stiles
# Date: 06/20/25

# Part 1: Longest Increasing Subsequence

from functools import lru_cache

# 1) Pure recursive
def lis_rec(arr, n):
    if n == 1:
        return 1
    best = 1
    for i in range(1, n):
        cur = lis_rec(arr, i)
        if arr[i-1] < arr[n-1] and cur + 1 > best:
            best = cur + 1
    return best

def lis_rec_driver(arr):
    return max(lis_rec(arr, i) for i in range(1, len(arr)+1))


# 2) Top-down with memoization
def lis_memo(arr):
    @lru_cache(None)
    def helper(k):
        best = 1
        for i in range(1, k):
            if arr[i-1] < arr[k-1]:
                best = max(best, helper(i) + 1)
        return best
    return max(helper(i) for i in range(1, len(arr)+1))


# 3) Bottom-up tabulation
def lis_tab(arr):
    n = len(arr)
    dp = [1]*n
    for i in range(n):
        for j in range(i):
            if arr[j] < arr[i]:
                dp[i] = max(dp[i], dp[j] + 1)
    return max(dp)


# Test
arr = [10, 22, 9, 33, 21, 50, 41, 60]
print("LIS (rec):", lis_rec_driver(arr))
print("LIS (memo):", lis_memo(arr))
print("LIS (tab):",  lis_tab(arr))


LIS (rec): 5
LIS (memo): 5
LIS (tab): 5


In [6]:
#Part 2: 0/1 Knapsack

from functools import lru_cache

wt  = [1, 3, 4, 5]
val = [1, 4, 5, 7]
W   = 7
n   = len(wt)

# 1) Pure recursive
def knap_rec(remaining, i):
    if i == 0 or remaining == 0:
        return 0
    if wt[i-1] > remaining:
        return knap_rec(remaining, i-1)
    return max(
        val[i-1] + knap_rec(remaining - wt[i-1], i-1),
        knap_rec(remaining, i-1)
    )

# 2) Top-down with memoization
@lru_cache(None)
def knap_memo(remaining, i):
    if i == 0 or remaining == 0:
        return 0
    if wt[i-1] > remaining:
        return knap_memo(remaining, i-1)
    return max(
        val[i-1] + knap_memo(remaining - wt[i-1], i-1),
        knap_memo(remaining, i-1)
    )

# 3) Bottom-up tabulation
def knap_tab(W, wt, val):
    m = len(wt)
    dp = [[0]*(W+1) for _ in range(m+1)]
    for i in range(1, m+1):
        for w in range(1, W+1):
            if wt[i-1] <= w:
                dp[i][w] = max(val[i-1] + dp[i-1][w-wt[i-1]], dp[i-1][w])
            else:
                dp[i][w] = dp[i-1][w]
    return dp[m][W]


# Test
print("Knapsack (rec):",  knap_rec(W, n))
print("Knapsack (memo):", knap_memo(W, n))
print("Knapsack (tab):",  knap_tab(W, wt, val))


Knapsack (rec): 9
Knapsack (memo): 9
Knapsack (tab): 9


In [7]:
# Part 3: Edit Distance

from functools import lru_cache

s1, s2 = "kitten", "sitting"

# 1) Pure recursive
def ed_rec(a, b):
    if not a: return len(b)
    if not b: return len(a)
    if a[-1] == b[-1]:
        return ed_rec(a[:-1], b[:-1])
    return 1 + min(
        ed_rec(a[:-1], b),    # delete
        ed_rec(a,    b[:-1]), # insert
        ed_rec(a[:-1], b[:-1])# replace
    )

# 2) Top-down with memoization
@lru_cache(None)
def ed_memo(i, j):
    if i == 0: return j
    if j == 0: return i
    if s1[i-1] == s2[j-1]:
        return ed_memo(i-1, j-1)
    return 1 + min(
        ed_memo(i-1, j),   # delete
        ed_memo(i,   j-1), # insert
        ed_memo(i-1, j-1)  # replace
    )

# 3) Bottom-up tabulation
def ed_tab(a, b):
    m, n = len(a), len(b)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m+1): dp[i][0] = i
    for j in range(n+1): dp[0][j] = j
    for i in range(1, m+1):
        for j in range(1, n+1):
            if a[i-1] == b[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
    return dp[m][n]


# Test
print("EditDist (rec):",  ed_rec(s1, s2))
print("EditDist (memo):", ed_memo(len(s1), len(s2)))
print("EditDist (tab):",  ed_tab(s1, s2))



EditDist (rec): 3
EditDist (memo): 3
EditDist (tab): 3


In [8]:
# Part 4: Compare recursive vs. tabulation methods & analyze time/space complexity



# General Differences

# Recursive (naïve):
# - Top-down exploration of subproblems via function calls
# - Recomputes the same subproblems many times
# - Exponential time in the worst case (e.g. O(2^n), O(3^min(m,n)))
# - Space usage is just the call stack (O(n) or O(m+n)), but risk of recursion-depth limits
# - Very straightforward to implement, but scales poorly

# Tabulation (bottom-up DP):
# - Builds up solutions from the smallest subproblems in a table/array
# - Stores and reuses all intermediate results, avoiding repeated work
# - Polynomial time (e.g. O(n^2), O(nW), O(mn))
# - Space proportional to the DP table (O(n), O(nW), or O(mn))
# - Eliminates recursion overhead and is often faster in practice, but needs more memory



# Problem-Specific Comparison

# 1. Longest Increasing Subsequence (LIS)
#   Recursive:
#     Brute-force search over subsequences
#     Time: O(2^n), Space: O(n) (call stack)
#   Tabulation:
#     Fill dp[i] = length of LIS ending at i
#     Time: O(n^2), Space: O(n)

# 2. 0/1 Knapsack
#   Recursive:
#     For each item, try include vs. exclude branches
#     Time: O(2^n), Space: O(n)
#   Tabulation:
#     Build table dp[i][w] for items 1..i and capacity w
#     Time: O(nW), Space: O(nW)

# 3. Edit Distance
#   Recursive:
#     Try delete/insert/replace at each mismatch
#     Time: O(3^{min(m,n)}), Space: O(m + n)
#   Tabulation:
#     Fill matrix dp[i][j] for prefixes of lengths i and j
#     Time: O(mn), Space: O(mn)

